---
title: EpigenomeXcan test on Br rats
author: Sabrina Mi
date: 7/10/2024
description: Calculate associations between predicted gene expression in adipose tissue and BMI to identify significant gene, while I figure out high to scale up predicting epigenome step.
---

CPU times: 

1. Compute founders matrix (n_genes, 8)
    * 3190 genes: ~110s
2. Compute samples matrix (n_samples, n_genes, 8):
    * 3190 genes, 1 sample: ~4s
    * 3190 genes, 10 samples: ~40s

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py
import os
import time
import bisect
columns = ['ACI', 'BN', 'BUF', 'F344', 'M520', 'MR', 'WKY', 'WN']

2024-07-25 04:12:42.617981: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 04:12:49.066611: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /soft/compilers/cudatoolkit/cuda-11.8.0/extras/CUPTI/lib64:/soft/compilers/cudatoolkit/cuda-11.8.0/lib64:/soft/libraries/trt/TensorRT-8.5.2.2.Linux.x86_64-gnu.cuda-11.8.cudnn8.6/lib:/soft/libraries/nccl/nccl_2.16.2-1+cuda11.8_x86_64/lib:/soft/libraries/cudnn/cudnn-11-linux-x64-v8.6.0.163/lib:/soft/compilers/cudatoolkit/cuda-11.8.0/extras/CUPTI/lib64:/soft/compilers/cudatoolkit/cuda-11.8.0/lib64:/so

In [21]:
gene_annot = pd.read_csv("/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes/gene_mapping.txt")
genes_by_chrom = gene_annot.groupby('chromosome')

In [4]:
probabilities_dir = "/eagle/AIHPC4Edu/sabrina/Br_genotype_probabilities"
reference_dir = "/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes/human"
output_dir = "/eagle/AIHPC4Edu/sabrina/Br_prediction_from_founders"

Realizing my haplotype probabilities storage is redundant, taking up extra space.

In [ ]:
#| code-fold: true
probabilities_file = f'{probabilities_dir}/chr1_probabilities.h5'
with h5py.File(probabilities_file, 'a') as hf:
    for dataset_name in list(hf.keys()):
        if hf[dataset_name].shape == 8:
            continue
        # Read the dataset
        data = hf[dataset_name][:]
        positions = data[:, 0]  # Assuming the positions column is the first column
        new_data = data[:, 1:]  # All columns except the first one
        
        # Create a temporary dataset without the positions column
        temp_dataset_name = f"temp_{dataset_name}"
        hf.create_dataset(temp_dataset_name, data=new_data)
        
        # Delete the original dataset
        del hf[dataset_name]
        
        # Rename the temporary dataset to the original dataset name
        hf.move(temp_dataset_name, dataset_name)
        
    hf.create_dataset('positions', data=positions)
        # Store the positions vector as metadata (attribute) for the dataset



split genes by chromosome number
for individual, query haplotype probabilities at each gene tss, so we return an 8 x n_gene matrix. by batching by genes n_genes should be in the 1-2K range. stack to return a 3d array with dimensions n_samples x 8 x n_gene
next query reference epigenome at each gene [446:450, CAGE_index], return 8 x n_gene matrix
matrix multiply with something like tf.transpose(tf.tensordot(_W, _X, axes=[[1],[1]]),[1,0,2])
https://stackoverflow.com/questions/41870228/understanding-tensordot

In [63]:
for chr, genes_df in genes_by_chrom:
    reference_file = f'{reference_dir}/{chr}_genes.h5'
    with h5py.File(reference_file, 'r') as ref:
        rows = []
        for gene in genes_df['gene']:
            founder_predictions = ref[gene][:, 446:450, 5278]
            rows.append(founder_predictions)
        ref_matrix = np.stack(rows, axis=0)
        ref_tensor = tf.reduce_mean(tf.convert_to_tensor(ref_matrix, dtype=tf.float32), axis=2)
        print(ref_tensor.shape)
    break

(3190, 8)


In [76]:
for chr, genes_df in genes_by_chrom:
    probabilities_file = f'{probabilities_dir}/{chr}_probabilities.h5'
    with h5py.File(probabilities_file, 'r') as prob:
        positions = prob['positions'][:]
        individuals = list(prob.keys())
        population_prob = []
        for sample in individuals[10:20]:
            rows = []
            for tss in genes_df['tss']:
                index = bisect.bisect_left(positions, tss)
                tss_prob = (prob[sample][index-1,:] + prob[sample][index,:]) / 2
                rows.append(tss_prob)
            sample_prob = np.vstack(rows)
            population_prob.append(sample_prob)
        prob_matrix = np.stack(population_prob, axis=0)
        prob_tensor = tf.convert_to_tensor(prob_matrix, dtype = tf.float32)
    break

In [74]:
prob_tensor.shape

TensorShape([10, 3190, 8])

In [75]:
ref_tensor.shape

TensorShape([3190, 8])